In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd
import sqlite3

PATH = 'chromedriver'

In [2]:
driver = webdriver.Chrome(PATH)

# Scraping Function

In [3]:
#takes a link, the table id, and the index of the header row, returns the data as a df
def get_data(url, findId, headerIndex):
    driver.get(url)
    res = driver.find_element(By.ID, findId)
    rows = res.find_elements(By.TAG_NAME, "tr")
    #get column names from first row
    cols = []
    for ele in rows[headerIndex].find_elements(By.TAG_NAME, "th"):
        cols.append(ele.text)
    cols[1:]
    #create dataframe and append table rows to it
    stats = pd.DataFrame(columns = cols[1:])
    for row in rows:
        player = []
        for td in row.find_elements(By.TAG_NAME, "td"):
            player.append(td.text)
        #try except is needed here so that header rows don't throw an error when added to the df and are instead skipped
        try:
            stats.loc[len(stats)] = player
        except:
            continue
    return(stats)

In [4]:
#Get Player data for all major stats
gsc_stats  = get_data('https://fbref.com/en/comps/22/gca/Major-League-Soccer-Stats', 'stats_gca', 1)
defense_stats = get_data('https://fbref.com/en/comps/22/defense/Major-League-Soccer-Stats', 'stats_defense', 1)
keeper_stats = get_data('https://fbref.com/en/comps/22/keepers/Major-League-Soccer-Stats', 'stats_keeper', 1)
misc_stats = get_data('https://fbref.com/en/comps/22/misc/Major-League-Soccer-Stats', 'stats_misc', 1)
pass_stats = get_data('https://fbref.com/en/comps/22/passing/Major-League-Soccer-Stats', 'stats_passing', 1)
possession_stats = get_data('https://fbref.com/en/comps/22/possession/Major-League-Soccer-Stats', 'stats_possession', 1)
shooting_stats = get_data('https://fbref.com/en/comps/22/shooting/Major-League-Soccer-Stats', 'stats_shooting', 1)
time_stats = get_data('https://fbref.com/en/comps/22/playingtime/Major-League-Soccer-Stats', 'stats_playing_time', 1)

In [8]:
#Get Team data for all major stats
east_squads = get_data('https://fbref.com/en/comps/22/Major-League-Soccer-Stats#all_stats_squads_standard', 'results2022221Eastern-Conference_overall', 0)
west_squads = get_data('https://fbref.com/en/comps/22/Major-League-Soccer-Stats#all_stats_squads_standard', 'results2022221Western-Conference_overall', 0)

squad_goalkeeping = get_data('https://fbref.com/en/comps/22/keepers/Major-League-Soccer-Stats', 'stats_squads_keeper_for', 1)
squad_gsc  = get_data('https://fbref.com/en/comps/22/gca/Major-League-Soccer-Stats', 'stats_squads_gca_for', 1)
squad_defense = get_data('https://fbref.com/en/comps/22/defense/Major-League-Soccer-Stats', 'stats_squads_defense_for', 1)
squad_misc = get_data('https://fbref.com/en/comps/22/misc/Major-League-Soccer-Stats', 'stats_squads_misc_for', 1)
squad_pass = get_data('https://fbref.com/en/comps/22/passing/Major-League-Soccer-Stats', 'stats_squads_passing_for', 1)
squad_possession = get_data('https://fbref.com/en/comps/22/possession/Major-League-Soccer-Stats', 'stats_squads_possession_for', 1)
squad_shooting = get_data('https://fbref.com/en/comps/22/shooting/Major-League-Soccer-Stats', 'stats_squads_shooting_for', 1)
squad_time = get_data('https://fbref.com/en/comps/22/playingtime/Major-League-Soccer-Stats', 'stats_squads_playing_time_for', 1)


In [9]:
#creates function to add squad name to each squad df instead of an id number, since there will never be duplicate team names
squad_dict = {0:'Atlanta Utd', 1: 'Austin FC', 2:'CF Montréal', 3:'Charlotte FC', 4: 'Chicago Fire', 5:'Colorado Rapids', 6:'Columbus Crew', 7:'D.C. United', 8:'FC Cincinnati', 9:'FC Dallas', 10:'Houston Dynamo', 11:'Inter Miami', 12:'LA Galaxy', 13:'Los Angeles FC', 14:'Minnesota Utd', 15:'Nashville', 16:'New England', 17:'NY Red Bulls', 18:'NYCFC', 19:'Orlando City', 20:'Philadelphia', 21:'Portland Timbers', 22:'Real Salt Lake', 23:'San Jose', 24:'Seattle', 25:'Sporting KC', 26:'Toronto FC', 27:'Vancouver'}

def add_squads(df):
    df.reset_index(inplace = True)
    df['squad'] = df['index'].apply(lambda x: squad_dict[x])
    df.drop('index', inplace = True, axis = 1)

In [10]:
add_squads(squad_goalkeeping)
add_squads(squad_gsc)
add_squads(squad_defense)
add_squads(squad_misc)
add_squads(squad_pass)
add_squads(squad_possession)
add_squads(squad_shooting)
add_squads(squad_time)

In [11]:
#Combine east and west team dfs to one big team df
squads = pd.concat([east_squads, west_squads]).reset_index(drop=True)

In [12]:
squads

,Squad,MP,W,D,L,GF,GA,GD,Pts,Pts/MP,xG,xGA,xGD,xGD/90,Last 5,Attendance,Top Team Scorer,Goalkeeper,Notes
0,Philadelphia,27,14,9,4,51,20,+31,51,1.89,46.2,30.8,+15.4,+0.57,W L W L W,"17,971",Dániel Gazdag - 13,Andre Blake,
1,CF Montréal,26,14,4,8,47,41,+6,46,1.77,39.9,26.9,+13.0,+0.50,D W D W W,"15,556",Romell Quioto - 14,Sebastian Breza,
2,NYCFC,26,13,6,7,48,30,+18,45,1.73,50.4,32.2,+18.2,+0.70,D L L L W,"17,414",Valentín Castellanos - 13,Sean Johnson,
3,NY Red Bulls,27,11,8,8,41,33,+8,41,1.52,38.9,33.1,+5.8,+0.21,L D L W D,"14,863",Lewis Morgan - 11,Carlos Coronel,
4,Orlando City,26,10,6,10,30,37,-7,36,1.38,29.6,35.2,-5.5,-0.21,L L L W W,"14,577",Ercan Kara - 8,Pedro Gallese,
5,Inter Miami,26,10,6,10,34,42,-8,36,1.38,33.1,37.6,-4.5,-0.17,D W D W W,"11,512","Gonzalo Higuaín, Leonardo Campana - 8",Drake Callender,
6,Columbus Crew,25,8,11,6,34,29,+5,35,1.40,32.8,31.5,+1.2,+0.05,D L W D D,"19,202","Lucas Zelarayán, Cucho - 8",Eloy Room,
7,New England,26,8,10,8,38,40,-2,34,1.31,34.2,38.2,-3.9,-0.15,D W W D L,"19,938","Adam Buksa, Gustavo Bou - 7",Đorđe Petrović,
8,FC Cincinnati,26,8,10,8,43,46,-3,34,1.31,40.4,39.9,+0.5,+0.02,D D W D D,"22,169",Brandon Vazquez - 15,Roman Celentano,
9,Charlotte,27,10,2,15,34,42,-8,32,1.19,30.6,37.0,-6.4,-0.24,W L L W L,"32,504",Karol Świderski - 8,Kristijan Kahlina,


# Player Database prep and creation

In [13]:
defense_stats.reset_index()

#Create dictionary of player ids so id can be used as a unique identifier for players, as 2 players could have the same name
player_ids = dict(zip(time_stats.Player, time_stats.index))

In [14]:
player_ids
#add ids to each player df

{'Paxten Aaronson': 0,
 'Lalas Abubakar': 1,
 'Nicolás Acevedo': 2,
 'Ifunanyachi Achara': 3,
 'Bryan Acosta': 4,
 'George Acosta': 5,
 'Kellyn Acosta': 6,
 'Luciano Acosta': 7,
 'Mo Adams': 8,
 'Samuel Adeniran': 9,
 'Harrison Afful': 10,
 'Samuel Afriyie Owusu': 11,
 'William Agada': 12,
 'Oskar Ågren': 13,
 'Daniel Aguirre': 14,
 'Ali Ahmed': 15,
 'Arnaud Ake': 16,
 'Tesho Akindele': 17,
 'Ayo Akinola': 18,
 'Jordy Alcivar': 19,
 'Caio Alexandre': 20,
 'Tony Alfaro': 21,
 'Noah Allen': 22,
 'Thiago Almada': 23,
 'Osvaldo Alonso': 24,
 'Jozy Altidore': 25,
 'Efrain Alvarez': 26,
 'Luis Amarilla': 27,
 'Frankie Amaya': 28,
 'Mikey Ambrose': 29,
 'Malte Amundsen': 30,
 'Max Anchor': 31,
 'Sebastian Anderson': 32,
 'José Andrés Martínez': 33,
 'Iván Angulo': 34,
 'Jalil Anibaba': 35,
 'Themi Antonoglou': 36,
 'Brian Anunga': 37,
 'Cristian Arango': 38,
 'César Araujo': 39,
 'Julian Araujo': 40,
 'Luiz Araújo': 41,
 'Daniel Armando Ríos': 42,
 'Adam Armour': 43,
 'Xavier Arreaga': 44,
 '

In [15]:
def set_id(df):
    df['player_id'] = df['Player'].apply(lambda x: player_ids[x])
    df.drop('Player', axis=1, inplace = True)



In [16]:
#Add Player ID to each table instead of player name
set_id(gsc_stats)
set_id(defense_stats)
set_id(keeper_stats)
set_id(pass_stats)
set_id(possession_stats)
set_id(shooting_stats)
set_id(misc_stats)
time_stats['player_id'] = time_stats['Player'].apply(lambda x: player_ids[x])
players = time_stats[['player_id', 'Player', 'Nation', 'Pos', 'Squad', 'Age', 'Born', '90s']]
time_stats.drop('Player', axis=1, inplace = True)

In [17]:
#Remove calculated and redundant fields from dataframes
gsc_stats.drop(['Nation', 'Pos', 'Squad', 'Age', 'Born', '90s', 'Matches', 'SCA', 'SCA90', 'GCA', 'GCA90'], axis =1, inplace = True)
defense_stats.drop(['Nation', 'Pos', 'Squad', 'Age', 'Born', '90s', 'Tkl%', '%', 'Matches', 'Tkl+Int'], axis=1, inplace = True)
keeper_stats.drop(['Nation', 'Pos', 'Squad', 'Age', 'Born', 'MP', 'Min', '90s', 'GA90', 'Save%', 'CS%', 'Matches', 'W', 'D', 'L'], axis=1, inplace = True)
pass_stats.drop(['Nation', 'Pos', 'Squad', 'Age', 'Born', '90s', 'Cmp%', 'Cmp%', 'Cmp%', 'Cmp%', 'A-xA', '1/3', 'Matches'], axis=1, inplace = True)
possession_stats.drop(['Nation', 'Pos', 'Squad', 'Age', 'Born', '90s', 'Touches', 'Succ%', '1/3', 'Rec%', 'Matches'], axis=1, inplace = True)
shooting_stats.drop(['Nation', 'Pos', 'Squad', 'Age', 'Born', '90s', 'SoT%', 'Sh/90', 'SoT/90', 'G/Sh', 'G/SoT', 'npxG/Sh', 'G-xG', 'np:G-xG', 'Matches'], axis=1, inplace = True)
misc_stats.drop(['Nation', 'Pos', 'Squad', 'Age', 'Born', '90s', 'Int', 'TklW', 'Won%', 'Matches'], axis=1, inplace = True)
time_stats.drop(['Nation', 'Pos', 'Squad', 'Age', 'Born', 'Mn/MP', 'Min%', '90s', '+/-', '+/-90', 'xG+/-', 'xG+/-90', 'Matches'], axis=1, inplace = True)

In [18]:
# Rename columns for database upload, drop calculated columns that weren't dropped earlier due to column name overlap
keeper_stats.rename(columns = {'Starts':'starts', 'GA':'ga', 'SoTA':'sota', 'Saves':'saves', 'CS':'cs', 'PKatt':'pkAtt', 'PKA':'pka', 'PKsv':'pksv', 'PKm':'pkm', 'player_id':'player_id'}, inplace = True)
gsc_stats.set_axis(['scPassLive', 'scPassDead', 'scDrib', 'scSh', 'scFld', 'scDef', 'gcPassLive', 'gcPassDead', 'gcDrib', 'gcSh', 'gcFld', 'gcDef', 'player_id' ], axis=1, inplace=True)
defense_stats.set_axis(['tkl', 'tklW', 'tklDefThird', 'tklMidThird', 'tklAttThird', 'tklVsDrib', 'tklAttVsDrib', 'past', 'press', 'pressW', 'pressDefThird', 'pressMidThird', 'pressAttThird', 'blk', 'blkSh', 'blkSot', 'blkPass', 'int', 'clr', 'err', 'player_id'], axis=1, inplace = True)
pass_stats.set_axis(['totCmp', 'totAtt', 'totDist', 'prgDist', 'shortCmp', 'shortAtt', 'medCmp', 'medAtt', 'lngCmp', 'lngAtt', 'ast', 'xA', 'kp', 'ppa', 'crsPa', 'progPasses', 'player_id' ], axis=1, inplace=True)
pass_stats.drop(['totCmp', 'totAtt'], axis = 1, inplace = True)
possession_stats.set_axis(['defPen', 'defThird', 'midThird', 'attThird', 'attPen', 'liveTouches', 'succDrib', 'attDrib', 'playersBeat', 'megs', 'carries', 'totDist', 'prgDist', 'progCarries', 'cpa', 'mis', 'tackled', 'passTarg', 'passRec', 'progPassRec', 'player_id'], axis=1, inplace=True)
shooting_stats.set_axis(['gls', 'sh', 'sot', 'avgDist', 'fk', 'pk', 'pkAtt', 'xG', 'npxG', 'player_id'], axis=1, inplace=True)
misc_stats.set_axis(['crdY', 'crdR', 'crdY2', 'fls', 'fld', 'off', 'crs', 'pkWon', 'PKcon', 'og', 'recov', 'airW', 'airL', 'player_id'], axis=1, inplace=True)
time_stats.set_axis(['mp', 'min', 'starts', 'mn/start', 'compl', 'subs', 'mn/sub', 'unSub', 'ppm', 'onG', 'onGa', 'on-off', 'onxG', 'onxGa', 'xon-off', 'player_id'], axis=1, inplace=True)
players.set_axis(['id', 'name', 'nation', 'pos', 'squad', 'age', 'born', '90s'], axis = 1, inplace = True)

In [59]:
#remove days from age, make it an int field
players['age'] = players['age'].apply(lambda x: x[0:2])
avg_age = int(players[players['age'] != ''].age.astype('int').mean())
players['age'] = players['age'].apply(lambda x: avg_age if x == '' else x)
players['age'] = players['age'].astype('int')

C:\Users\Brian\AppData\Local\Temp/ipykernel_50252/1609455280.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  players['age'] = players['age'].apply(lambda x: avg_age if x == '' else x)
C:\Users\Brian\AppData\Local\Temp/ipykernel_50252/1609455280.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  players['age'] = players['age'].astype('int')


In [61]:
#For players without an age impute the avg age
avg_born = players[players['born'] != ''].born.astype('int').mean()
players['born'] = players['born'].apply(lambda x: avg_born if x == '' else x)
players['born'] = players['born'].astype('int')

C:\Users\Brian\AppData\Local\Temp/ipykernel_50252/3349463930.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  players['born'] = players['born'].apply(lambda x: avg_born if x == '' else x)
C:\Users\Brian\AppData\Local\Temp/ipykernel_50252/3349463930.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  players['born'] = players['born'].astype('int')


0      2003
1      1994
2      1999
3      1997
4      1993
       ... 
842    2003
843    1998
844    1992
845    1986
846    2002
Name: born, Length: 847, dtype: int32

In [109]:
#convert on-offs to floats
time_stats['on-off'] = time_stats['on-off'].apply(lambda x: 0 if x == '' else x)
time_stats['on-off'] = time_stats['on-off'].astype('float')

time_stats['xon-off'] = time_stats['xon-off'].apply(lambda x: 0 if x == '' else x)
time_stats['xon-off'] = time_stats['xon-off'].astype('float')

time_stats

,mp,min,starts,mn/start,compl,subs,mn/sub,unSub,ppm,onG,onGa,on-off,onxG,onxGa,xon-off,player_id
0,18,377,2,69,0,16,15,5,2.06,4,3,-1.08,5.6,5.0,-0.48,0
1,25,"1,974",23,84,19,2,18,0,1.28,28,29,0.45,33.3,25.0,0.48,1
2,24,"1,680",20,77,7,4,33,1,1.83,32,18,0.20,35.5,19.9,0.48,2
3,10,272,0,,0,10,27,12,0.60,4,3,0.66,4.0,4.3,0.50,3
4,20,"1,477",17,83,10,3,21,3,1.35,27,26,0.48,26.1,21.1,0.00,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
842,4,114,1,45,0,3,23,5,0.25,0,4,-3.56,1.2,1.4,-0.73,842
843,9,123,1,90,1,8,4,16,1.56,0,3,-2.39,1.4,3.2,-1.75,843
844,24,"1,653",18,89,15,6,9,1,1.17,33,33,0.12,27.9,29.2,0.09,844
845,18,"1,412",17,82,13,1,13,0,0.94,21,27,0.77,17.3,21.7,0.36,845


In [110]:
#add to db next
#connect to mlsdb and create/update players table
with sqlite3.connect('mlsdb.sqlite') as conn:
    players.to_sql('players', conn, if_exists='replace', index = False, dtype = {'id':'INTEGER', 'name':'TEXT', 'nation':'TEXT', 'pos':'TEXT', 'squad':'TEXT', 'age':'INTEGER', 'born':'INTEGER', '90s':'REAL'})
    keeper_stats.to_sql('playerKeepers', conn, if_exists='replace', index = False, dtype={'starts':'INTEGER', 'ga':'INTEGER', 'sota':'INTEGER', 'saves':'INTEGER', 'cs':'INTEGER', 'ppkAtt':'INTEGER', 'pka':'INTEGER', 'pksv':'INTEGER', 'pkm':'INTEGER', 'player_id':'INTEGER'})
    defense_stats.to_sql('playerDefense', conn, if_exists='replace', index = False, dtype={'tkl':'INTEGER', 'tklW':'INTEGER', 'tklDefThird':'INTEGER', 'tklMidThird':'INTEGER', 'tklAttThird':'INTEGER', 'tklVsDrib':'INTEGER', 'tklAttVsDrib':'INTEGER', 'past':'INTEGER', 'press':'INTEGER', 'pressW':'INTEGER', 'pressDefThird':'INTEGER', 'pressMidThird':'INTEGER', 'pressAttThird':'INTEGER', 'blk':'INTEGER', 'blkSh':'INTEGER', 'blkSot':'INTEGER', 'blkPass':'INTEGER', 'int':'INTEGER', 'clr':'INTEGER', 'err':'INTEGER', 'player_id':'INTEGER'}) 
    gsc_stats.to_sql('playerGsc', conn, if_exists='replace', index = False, dtype = {'scPassLive':'INTEGER', 'scPassDead':'INTEGER', 'scDrib':'INTEGER', 'scSh':'INTEGER', 'scFld':'INTEGER', 'scDef':'INTEGER','gcPassLive':'INTEGER', 'gcPassDead':'INTEGER', 'gcDrib':'INTEGER', 'gcSh':'INTEGER', 'gcFld':'INTEGER', 'gcDef':'INTEGER', 'player_id':'INTEGER'})
    misc_stats.to_sql('playerMisc', conn, if_exists='replace', index = False, dtype={'crdY':'INTEGER', 'crdR':'INTEGER', 'crdY2':'INTEGER', 'fls':'INTEGER', 'fld':'INTEGER', 'off':'INTEGER', 'crs':'INTEGER', 'pkWon':'INTEGER', 'PKcon':'INTEGER','og':'INTEGER', 'recov':'INTEGER', 'airW':'INTEGER', 'airL':'INTEGER', 'player_id':'INTEGER'})  
    pass_stats.to_sql('playerPassing', conn, if_exists='replace', index = False, dtype={'totDist':'INTEGER', 'prgDist':'INTEGER', 'shortCmp':'INTEGER', 'shortAtt':'INTEGER', 'medCmp':'INTEGER', 'medAtt':'INTEGER','lngCmp':'INTEGER', 'lngAtt':'INTEGER', 'ast':'INTEGER', 'xA':'INTEGER', 'kp':'INTEGER', 'ppa':'INTEGER', 'crsPa':'INTEGER', 'progPasses':'INTEGER','player_id':'INTEGER'})
    possession_stats.to_sql('playerPossession', conn, if_exists='replace', index = False, dtype={'defPen':'INTEGER', 'defThird':'INTEGER', 'midThird':'INTEGER', 'attThird':'INTEGER', 'attPen':'INTEGER', 'liveTouches':'INTEGER', 'succDrib':'INTEGER', 'attDrib':'INTEGER', 'playersBeat':'INTEGER', 'megs':'INTEGER', 'carries':'INTEGER', 'totDist':'INTEGER', 'prgDist':'INTEGER', 'progCarries':'INTEGER', 'cpa':'INTEGER', 'mis':'INTEGER', 'tackled':'INTEGER', 'passTarg':'INTEGER', 'passRec':'INTEGER', 'progPassRec':'INTEGER', 'player_id':'INTEGER'})
    shooting_stats.to_sql('playerShooting', conn, if_exists='replace', index = False, dtype={'gls':'INTEGER', 'sh':'INTEGER', 'sot':'INTEGER', 'avgDist':'INTEGER', 'fk':'INTEGER', 'pk':'INTEGER', 'pkAtt':'INTEGER', 'xG':'INTEGER', 'npxG':'INTEGER', 'player_id':'INTEGER'})    
    time_stats.to_sql('playerTime', conn, if_exists='replace', index = False, dtype = {'mp':'INTEGER', 'min':'INTEGER', 'starts':'INTEGER', 'mn/start':'INTEGER', 'compl':'INTEGER', 'subs':'INTEGER', 'mn/sub':'INTEGER', 'unSub':'INTEGER','ppm':'REAL', 'onG':'INTEGER', 'onGa':'INTEGER', 'on-off':'REAL', 'onxG':'REAL', 'onxGa':'REAL', 'xon-off':'REAL', 'player_id':'INTEGER'}) 

# Squad Database Prep and Creation

In [20]:
#we don't need to set ID here as squad name is an adaquate primary key since it will never be duplicated
squads.drop(['Pts/MP', 'Pts', 'GD', 'GF', 'xGD/90', 'xGD', 'Top Team Scorer', 'Notes', 'xG'], axis=1, inplace = True)
squad_goalkeeping.drop(['# Pl', 'MP', 'Starts', 'Min', '90s', 'GA', 'GA90', 'Save%', 'W', 'D', 'L', 'CS%'], axis = 1, inplace = True)
squad_gsc.drop(['# Pl', '90s', 'SCA90', 'GCA90', 'SCA', 'GCA'], axis = 1, inplace = True)
#drop first tkl after renaming
squad_defense.drop(['# Pl', '90s', 'Tkl%', '%', 'Tkl+Int'], axis = 1, inplace = True)
squad_misc.drop(['# Pl', '90s', 'Int', 'TklW', 'Won%'], axis = 1, inplace = True)
#drop first cmp after renaming
squad_pass.drop(['# Pl', '90s', 'Cmp%', 'A-xA', '1/3'], axis = 1, inplace = True)
#drop first # pl after renaming
squad_possession.drop([ '90s', 'Touches', 'Succ%', '1/3', 'Rec%'], axis = 1, inplace = True)
squad_shooting.drop(['# Pl', '90s', 'SoT%', 'Sh/90', 'SoT/90', 'G/Sh', 'G/SoT', 'npxG/Sh', 'G-xG', 'np:G-xG'], axis = 1, inplace = True)
squad_time.drop(['# Pl', 'MP', 'Mn/MP', 'Min%', '90s', 'Starts', 'Mn/Start', 'Compl', 'PPM', '+/-', '+/-90', 'xG+/-', 'xG+/-90', 'onG', 'onGA', 'onxG', 'onxGA'], axis = 1, inplace = True)

In [21]:
squads.set_axis(['squad', 'mp', 'w', 'd', 'l', 'ga', 'xGA', 'last5', 'attendence', 'goalkeeper'], axis=1, inplace=True)
squad_goalkeeping.set_axis(['sota', 'saves', 'cs', 'pkAtt', 'pkA', 'pkSv', 'pkM', 'squad'], axis=1, inplace=True)
squad_defense.set_axis(['tkl', 'tklW', 'tklDefThird', 'tklMidThird', 'tklAttThird', 'tklVsDrib', 'attVsDrib', 'past', 'press', 'pressW', 'pressDefThird', 'pressMidThird', 'pressAttThird', 'blk', 'blkSh', 'blkSot', 'blkPass', 'int', 'clr', 'err', 'squad'], axis=1, inplace=True)
squad_misc.set_axis(['crdY', 'crdR', 'crdY2', 'fls', 'fld', 'off', 'crs', 'pkWon', 'pkCon', 'og', 'recov', 'airW', 'airL', 'squad'], axis=1, inplace=True)
squad_pass.set_axis(['totCmp', 'totAtt', 'totDist', 'prgDist', 'shortCmp', 'shortAtt', 'medCmp', 'medAtt', 'lngCmp', 'lngAtt', 'ast', 'xA', 'kp', 'ppa', 'crsPa', 'progPasses', 'squad'], axis=1, inplace=True)
squad_possession.set_axis(['# Pl', 'poss', 'defPen', 'defThird', 'midThird', 'attThird', 'attPen', 'live', 'dribW', 'dribAtt', 'playersBeat', 'megs', 'carries', 'totDist', 'prgDist', 'progCarries', 'cpa', 'mis', 'dis', 'targ', 'rec', 'progRec', 'squad' ], axis=1, inplace=True)
squad_possession.drop('# Pl', axis = 1, inplace = True)
squad_shooting.set_axis(['gls', 'sh', 'sot', 'avgDist', 'fk', 'pkW', 'pkAtt', 'xG', 'npxG', 'squad'], axis=1, inplace=True)
squad_time.set_axis(['age','min','subs','mn/sub','unsub', 'squad'], axis=1, inplace=True)
squad_gsc.set_axis(['scPassLive', 'scPassDead', 'scDrib', 'scSh', 'scFld', 'scDef', 'gcPassLive', 'gcPassDead', 'gcDrib', 'gcSh', 'gcFld', 'gcDef', 'squad' ], axis=1, inplace=True)


In [143]:
#add to db next
#connect to mlsdb and create/update players table
with sqlite3.connect('mlsdb.sqlite') as conn:
    squads.to_sql('squads', conn, if_exists='replace', index = False, dtype={'squad':'TEXT', 'mp':'INTEGER', 'w':'INTEGER', 'd':'INTEGER', 'l':'INTEGER', 'ga':'INTEGER', 'xGA':'REAL', 'last5':'TEXT', 'attendence':'INTEGER', 'goalkeeper':'TEXT'})
    squad_goalkeeping.to_sql('squadKeepers', conn, if_exists='replace', index = False, dtype={'sota':'INTEGER', 'saves':'INTEGER', 'cs':'INTEGER', 'pkAtt':'INTEGER', 'pkA':'INTEGER', 'pkSv':'INTEGER', 'pkM':'INTEGER', 'squad':'TEXT'})
    squad_defense.to_sql('squadDefense', conn, if_exists='replace', index = False, dtype= {'tkl':'INTEGER', 'tklW':'INTEGER', 'tklDefThird':'INTEGER', 'tklMidThird':'INTEGER', 'tklAttThird':'INTEGER', 'tklVsDrib':'INTEGER', 'attVsDrib':'INTEGER', 'past':'INTEGER', 'press':'INTEGER', 'pressW':'INTEGER', 'pressDefThird':'INTEGER', 'pressMidThird':'INTEGER', 'pressAttThird':'INTEGER', 'blk':'INTEGER', 'blkSh':'INTEGER', 'blkSot':'INTEGER', 'blkPass':'INTEGER', 'int':'INTEGER', 'clr':'INTEGER', 'err':'INTEGER', 'squad':'TEXT'}) 
    squad_gsc.to_sql('squadGsc', conn, if_exists='replace', index = False, dtype = {'scPassLive':'INTEGER', 'scPassDead':'INTEGER', 'scDrib':'INTEGER', 'scSh':'INTEGER', 'scFld':'INTEGER', 'scDef':'INTEGER','gcPassLive':'INTEGER', 'gcPassDead':'INTEGER', 'gcDrib':'INTEGER', 'gcSh':'INTEGER', 'gcFld':'INTEGER', 'gcDef':'INTEGER', 'squad':'TEXT'})
    squad_misc.to_sql('squadMisc', conn, if_exists='replace', index = False, dtype={'crdY':'INTEGER', 'crdR':'INTEGER', 'crdY2':'INTEGER', 'fls':'INTEGER', 'fld':'INTEGER', 'off':'INTEGER', 'crs':'INTEGER', 'pkWon':'INTEGER', 'pkCon':'INTEGER', 'og':'INTEGER', 'recov':'INTEGER', 'airW':'INTEGER', 'airL':'INTEGER', 'squad':'REAL'})  
    squad_pass.to_sql('squadPassing', conn, if_exists='replace', index = False, dtype = {'totCmp':'INTEGER', 'totAtt':'INTEGER', 'totDist':'INTEGER', 'prgDist':'INTEGER', 'shortCmp':'INTEGER', 'shortAtt':'INTEGER', 'medCmp':'INTEGER', 'medAtt':'INTEGER', 'lngCmp':'INTEGER', 'lngAtt':'INTEGER', 'ast':'INTEGER', 'xA':'INTEGER', 'kp':'INTEGER', 'ppa':'INTEGER', 'crsPa':'INTEGER', 'progPasses':'INTEGER', 'squad':'TEXT'})
    squad_possession.to_sql('squadPossession', conn, if_exists='replace', index = False, dtype={'poss':'INTEGER', 'defPen':'INTEGER', 'defThird':'INTEGER', 'midThird':'INTEGER', 'attThird':'INTEGER', 'attPen':'INTEGER', 'live':'INTEGER', 'dribW':'INTEGER', 'dribAtt':'INTEGER', 'playersBeat':'INTEGER', 'megs':'INTEGER', 'carries':'INTEGER', 'totDist':'INTEGER','prgDist':'INTEGER', 'progCarries':'INTEGER', 'cpa':'INTEGER', 'mis':'INTEGER', 'dis':'INTEGER', 'targ':'INTEGER', 'rec':'INTEGER', 'progRec':'INTEGER','squad':'TEXT'})
    squad_shooting.to_sql('squadShooting', conn, if_exists='replace', index = False, dtype={'gls':'INTEGER', 'sh':'INTEGER', 'sot':'INTEGER', 'avgDist':'REAL', 'fk':'INTEGER', 'pkW':'INTEGER', 'pkAtt':'INTEGER', 'xG':'REAL', 'npxG':'REAL', 'squad':'TEXT'})    
    squad_time.to_sql('squadTime', conn, if_exists='replace', index = False, dtype={'age':'REAL', 'min':'INTEGER', 'subs':'INTEGER', 'mn/sub':'INTEGER', 'unsub':'INTEGER', 'squad':'TEXT'}) 